<a href="https://colab.research.google.com/github/Abrazacs/rps_kaggle/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Environment_example_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -q -U kaggle_environments

In [25]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [26]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def rock_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Создаем агента, который всегда выдает играет только "бумагой" - 1

In [27]:
%%writefile papper_agent.py
def papper_agent(observation, configuration):
    return 1

Overwriting papper_agent.py


Создаем агента, который всегда выдает играет только "ножницами" - 2

In [28]:
%%writefile scissors_agent.py
def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


Создаем агента, который использует случайное значение всегда

In [29]:
%%writefile random_choose_agent.py
def rando_choose_agent(observation, configuration):
    return random.randrange(0, 3)

Overwriting random_choose_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [30]:
%%writefile copy_opponent.py
#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, 3)

Overwriting copy_opponent.py


Создаем агента, который делает выбор последовательно камень -> ниожницы -> бумага -> камень и т.д.

In [31]:
%%writefile step_by_step_agent.py

def step_by_step(observation, configuration):
  #просто берем остаток от деления на 3, полученное число и будет нашим ходом
  return observation.step % 3


Overwriting step_by_step_agent.py


Создаем агента, который выбирает контр пик к последнему действию оппонента

In [32]:
%%writefile counter_pick_agent.py

#в соответствии с rps.json lastOpponentAction - это число от 0 до 4. Агент будет выбирать контр пик основываясь на последнем ходе оппонента.
#Т.к.контр пик на 1 больше предыдущего выбора оппонента, то достаточно к последнему выбору оппонента прибавлять 1 и получи искомое значение,
#а если перевалим за 2, то конрт пик = 0

def counter_pick(observation, configuration):
  counter_pick = 0 # если у противника не было еще ходов, то выбираем 0 по дефолту
  if observation.step > 0:
    counter_pick = observation.lastOpponentAction + 1
  if counter_pick > 2:
    counter_pick = 0
  return counter_pick

Overwriting counter_pick_agent.py


Создаем агента, который выбирает ход, приводящий к проигрышу, по последнему действию оппонента

In [33]:
%%writefile loose_pick_agent.py

def loose_pick(observation, configuration):
  loose_pick = 0 # если у противника не было еще ходов, то выбираем 0 по дефолту
  if observation.step > 0:
    loose_pick = observation.lastOpponentAction + 2
  # В случае, если оппонент выбрал на прошлом ходу бумагу или ножницы, то наш выбор будет равен 3 или 4,
  # соответсвенно в данном случае надо брать остаток от деления на 3 (получим 0 - камень или 1 - бумага)
  if loose_pick > 2:
    loose_pick = loose_pick % 3
  return loose_pick

Overwriting loose_pick_agent.py


Создаем агента, который никогда не ходит камнем

In [34]:
%%writefile not_rock_agent.py

def not_rock_agent(observation, configuration):
  return random.randrange(1, 3)

Overwriting not_rock_agent.py


Создаем агента, который никогда не ходит бумагой

In [35]:
%%writefile not_papper_agent.py
def not_papper_agent(observation, configuration):
  # единица никогда не будет выбрана, т.к. шаг 2
  return random.randrange(0, 3, 2)


Overwriting not_papper_agent.py


Создаем агента, который никогда не ходит ножницами

In [36]:
%%writefile not_scissors_agent.py

def not_scissors_agent(observation, configuration):
  return random.randrange(0, 2)

Overwriting not_scissors_agent.py


Агент, который случайно выбирает шаг, но так, чтобы разница между его самым частым выбором и самым редким не превышала 3

In [37]:
%%writefile random_v1_agent.py

steps_history = {0:0,1:0,2:0}

def random_v1_agent(observation, configuration):
    global steps_history
    min_val = min(steps_history.values()) # смотрим минимальное значение
    max_val = max(steps_history.values()) # смотрим максимальное значание
    if max_val - min_val > 3:
        for i in steps_history:
            if steps_history[i] == min_val: # если разница больше 3, то ищем самый редкий выбор и возвращаем его, при этом увеличиваем кол-во его выборов
                steps_history[i] += 1
                return i
    else:
        result =random.randrange(0,3) # если разница между min_val и max_val не больше 3, то делаем случайный выбор
        steps_history[result] += 1
        return result


Overwriting random_v1_agent.py


Агент, который пытается пердугадать ход исходя и самого частого хода оппонента

In [38]:
%%writefile enemy_common_pick_agent.py

enemy_picks = {
    0:0,
    1:0,
    2:0,
}

def enemy_common_pick_agent(observation, configuration):
  global enemy_picks

  # Если это не первый шаг, то заполняем статистику по шагам оппонента
  if observation.step > 0:
    last_action = observation.lastOpponentAction
    enemy_picks[last_action] += 1

  # получаем самый частый выбор опоненат, если есть с одинаковой частотой выбора, то берем первый попавшийся.
  common_pick = next(filter(lambda x: enemy_picks[x] == max(enemy_picks.values()), enemy_picks.keys()))
  counter_pick = common_pick+1
  return 0 if counter_pick > 2 else counter_pick

Overwriting enemy_common_pick_agent.py


Будем исходить из того, что выбор камня, ножницы или бумаги являются равновероятностными событиями. В данном случае наш агент будет выбирать контр пик к действию, которое наш оппонент выбирал реже всего, т.к. вероятность его наступления в этом случае должна быть самой большой.

In [39]:
%%writefile uncomon_enemy_action_agent.py

statistics = {
    0:0,
    1:0,
    2:0,
}

def uncomon_enemy_action_agent(observation, configuration):
  global statistics

  # Если это не первый шаг, то заполняем статистику по шагам оппонента
  if observation.step > 0:
    last_action = observation.lastOpponentAction
    statistics[last_action] += 1

  # получаем самый редкий выбор опоненат, если есть с одинаковой частотой выбора, то берем первый попавшийся.
  uncommon_pick = next(filter(lambda x: enemy_picks[x] == min(enemy_picks.values()), enemy_picks.keys()))
  counter_pick = uncommon_pick+1
  return 0 if counter_pick > 2 else counter_pick


Overwriting uncomon_enemy_action_agent.py


Турнир, каждый играет со всеми:


In [40]:
agents_lst = ["rock_agent.py",
              "scissors_agent.py",
              "copy_opponent.py",
              "step_by_step_agent.py",
              "counter_pick_agent.py",
              "papper_agent.py",
              "random_choose_agent.py",
              "loose_pick_agent.py",
              "not_rock_agent.py",
              "not_papper_agent.py",
              "not_scissors_agent.py",
              "random_v1_agent.py",
              "enemy_common_pick_agent.py",
              "uncomon_enemy_action_agent.py"]

for agent in agents_lst:
  for second_agent in agents_lst:
    if agent != second_agent:
      print(f"Играет {agent} и {second_agent}")
      print("Результат:", evaluate("rps",[agent, second_agent], configuration={"episodeSteps": 100}))

Играет rock_agent.py и scissors_agent.py
Результат: [[99.0, -99.0]]
Играет rock_agent.py и copy_opponent.py
Результат: [[1, None]]
Играет rock_agent.py и step_by_step_agent.py
Результат: [[0, 0]]
Играет rock_agent.py и counter_pick_agent.py
Результат: [[-98.0, 98.0]]
Играет rock_agent.py и papper_agent.py
Результат: [[-99.0, 99.0]]
Играет rock_agent.py и random_choose_agent.py
Результат: [[1, None]]
Играет rock_agent.py и loose_pick_agent.py
Результат: [[98.0, -98.0]]
Играет rock_agent.py и not_rock_agent.py
Результат: [[1, None]]
Играет rock_agent.py и not_papper_agent.py
Результат: [[1, None]]
Играет rock_agent.py и not_scissors_agent.py
Результат: [[1, None]]
Играет rock_agent.py и random_v1_agent.py
Результат: [[1, None]]
Играет rock_agent.py и enemy_common_pick_agent.py
Результат: [[-99.0, 99.0]]
Играет rock_agent.py и uncomon_enemy_action_agent.py
Результат: [[1, None]]
Играет scissors_agent.py и rock_agent.py
Результат: [[-99.0, 99.0]]
Играет scissors_agent.py и copy_opponent.py